In [ ]:
!ls ../input/cassava-leaf-disease-classification/

In [ ]:
#Much of the code used here is based on: https://keras.io/examples/keras_recipes/tfrecord/
#More code taken from my work on the MARCO crystallization image dataset

import pandas as pd
import numpy as np
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid

import glob
import os

from PIL import Image

from keras.models import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

from functools import partial

# Tuning and training params:
BATCH_SIZE = 1
IMAGE_SIZE = [512, 512]
epochs = 18

csv_df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
csv_df.head(10)

In [ ]:
#Our data is unbalanced. We'll have to remedy that or all the predictions will be "3"
sns.distplot(csv_df['label'])

In [ ]:
#Getting weights to account for unbalanced data
# weight = 1/(number of images in class)*(size of dataset)/(number of classes)
weights = [(1/csv_df['label'].value_counts()[n])*csv_df['label'].shape[0]/5 for n in csv_df['label'].unique()]
weights = dict(zip(list(range(0,5)),weights))

![](http://)The data here provided is also provided in tfrec format which speeds up training a good deal.

In [ ]:
train_dir = '/kaggle/input/cassava-leaf-disease-classification/train_tfrecords/*'
test_dir = '/kaggle/input/cassava-leaf-disease-classification/test_tfrecords/*'

train_list = glob.glob(train_dir)
test_list = glob.glob(test_dir)
print("train: " + str(len(train_list)) + "\ntest: " + str(len(test_list)))

# Building a data generator

To get tfrecords into a neural-network for training we need to first build a data-generator to make batches of images (which fit in our RAM)
We need a few steps to get there.
1. What features are in the TFRecords? We need to unpack them and see what features are in there.
2. We need a getter to give us shuffled training batches that don't overwhelm the ram.
3. The getter needs a map function to read the features out of the dataset.
4. The map function needs an image decoder to do our image manipulations for us.
5. The image decoder should have an option to perform augmentation operations.

In [ ]:
#To get some idea what features are in the TFRecords, we need to unpack them.
# Found this code on stackexchange posted by user jdehesa

def list_record_features(tfrecords_path):
    # Dict of extracted feature information
    features = {}
    # Iterate records
    for rec in tf.data.TFRecordDataset([str(tfrecords_path)]):
        # Get record bytes
        example_bytes = rec.numpy()
        # Parse example protobuf message
        example = tf.train.Example()
        example.ParseFromString(example_bytes)
        # Iterate example features
        for key, value in example.features.feature.items():
            # Kind of data in the feature
            kind = value.WhichOneof('kind')
            # Size of data in the feature
            size = len(getattr(value, kind).value)
            # Check if feature was seen before
            if key in features:
                # Check if values match, use None otherwise
                kind2, size2 = features[key]
                if kind != kind2:
                    kind = None
                if size != size2:
                    size = None
            # Save feature data
            features[key] = (kind, size)
    return features

record_features = list_record_features(glob.glob(train_dir+"*")[0])
print(record_features)

In [ ]:
feature_description = {
    "image" : tf.io.VarLenFeature(tf.string), #image jpg encoded
    "image_name" : tf.io.FixedLenFeature([], tf.string), #image unique identifier
    "target" : tf.io.FixedLenFeature([], tf.int64), #Target class
}

AUTOTUNE = tf.data.experimental.AUTOTUNE

# I originally got this code from https://keras.io/examples/keras_recipes/tfrecord/ - Light customizations as needed.

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(buffer_size = 2048, seed = 42)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset


def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order) # again, faster.
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset


def read_tfrecord(example, labeled=True):
    if labeled:
        tfrecord_format = (
            {
                "image": tf.io.VarLenFeature(tf.string),
                "target": tf.io.FixedLenFeature([], tf.int64),
                "image_name": tf.io.FixedLenFeature([], tf.string)
            })
    else:
        tfrecord_format = ({
            "image": tf.io.FixedLenFeature([], tf.string),
            "image_name": tf.io.FixedLenFeature([], tf.string)
        })
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"], labeled)
    name = example["image_name"]
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        label = tf.one_hot(label, depth = 5)
        return image, label
    else:
        return image, name


def decode_image(image, labeled):
    if labeled:
        image = tf.image.decode_jpeg(image.values[0], channels=3)
        image = augment(image)
    else:
        image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_crop(image, size=[*IMAGE_SIZE, 3])
    #TODO: Add augmentation. MixUp and CutUp from https://www.kaggle.com/markwijkhuizen/tf-efficientnetb4-mixup-and-cutmix-cv-0-90
    return image

In [ ]:
def lrfn(epoch, bs=BATCH_SIZE, EPOCHS=epochs):
    # Config
    LR_START = 1e-6
    LR_MAX = 2e-4
    LR_FINAL = 1e-6
    LR_RAMPUP_EPOCHS = epochs/10
    LR_SUSTAIN_EPOCHS = 0
    DECAY_EPOCHS = epochs  - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
    LR_EXP_DECAY = (LR_FINAL / LR_MAX) ** (1 / (EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1))

    if epoch < LR_RAMPUP_EPOCHS: # exponential warmup
        lr = LR_START + (LR_MAX + LR_START) * (epoch / LR_RAMPUP_EPOCHS) ** 2.5
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS: # sustain lr
        lr = LR_MAX
    else: # cosine decay
        epoch_diff = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        decay_factor = (epoch_diff / (DECAY_EPOCHS+0.01)) * 3.14
        decay_factor= (tf.math.cos(decay_factor).numpy() + 1) / 2        
        lr = LR_FINAL + (LR_MAX - LR_FINAL) * decay_factor

    return lr

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "cassava_model_checkpoint.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

lr_cb = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=1)



# Ready for training
Batch size - we'll do 3, it's what fits in ram. If we decrease the image resolution we'll turn that up.

In [ ]:
# Tuning and training params:
BATCH_SIZE = 3
IMAGE_SIZE = [299, 299]
epochs = 10

print("Train Files: ", len(train_list))
print("Test Files: ", len(test_list))

train_dataset = get_dataset(train_list[1])
valid_dataset = get_dataset(train_list[0])
test_dataset = get_dataset(test_list, labeled=False)

# Inception V4 
It's not added to keras yet so I added it here as a utility script. IV4 may not be the best choice for this competition since it's very computationally intensive but it has the best balance of time-to-train and performance across multiple tasks as compared to other model architectures so I'm using it here.

In [ ]:
from inceptionv4 import create_model

In [ ]:
base_model = create_model(
    num_classes=5,
)

base_model.trainable = True
inputs = tf.keras.Input(shape=([*IMAGE_SIZE, 3]))
x = base_model(inputs, training = True)
#x = tf.keras.layers.Dropout(0.4)(x)
#x = tf.keras.layers.Dense(2000, activation='relu')(x)
#x = tf.keras.layers.Dense(500, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy,
              metrics=tf.keras.metrics.categorical_accuracy)

In [ ]:
epochs = 10
model.fit(
    train_dataset,
    #validation_data = valid_dataset,
    batch_size = 8,
    epochs = epochs,
    callbacks = [lr_cb, checkpoint_cb, early_stopping_cb],
    class_weight = weights
)
model.save("./Cassava-Model.h5")



In [ ]:
y_pred = model.predict(valid_dataset)

In [ ]:
sns.distplot(y_pred.argmax(axis=1))

In [ ]:
labels = []
ids = []
for n in test_dataset:
    name = (str(n[1].numpy()[0]).replace('b', "").replace("'", ""))
    pred = model.predict(n[0]).argmax()
    ids.append(name)
    labels.append(pred)

In [ ]:
preds = pd.DataFrame.from_dict({"image_id":ids, "label":labels})
preds.to_csv("submission.csv")

# It's still overfitting and I'm out of time.
As much as I hate to give up, it's time to turn in.
See you next kernel.

In [ ]:
preds